DSCI 100: Individual Project

In [19]:
#(1) Data Description

The project uses two related datasets, players.csv and sessions.csv

players.csv: A list of all unique players, including data about each player.

sessions.csv: A list of individual play sessions by each player, including data about the session.

players.csv

Number of observations(rows): 196

Number of variables(columns): 7

| Variable Name | Type | Description |
| ------------- | ---- | ----------- |
| Experience | chr | Describing a player’s experience level |
| Subscribe | lgl | Player subscribed or not to the research server | 
| HashedEmail | chr | Representing an anonymized player ID |
| Played_hours | dbl | How many total hours the player has played |
| Name | chr | Player’s nickname in game |
| Gender | chr | Player's gender |
| Age | dbl | Player’s age in years |

sessions.csv

Number of observations(rows): 1535

Number of variables(columns): 5

| Variable Name | Type | Description |
| ------------- | ---- | ----------- |
| hashedEmail | chr | Representing an anonymized player ID |
| start_time | chr | Time when the session was created(human-readable) |
| end_time | chr | Time when the session was end(human-readable) |
| original_start_time | dbl | Unix timestamp (milliseconds) of session start |
| original_end_time | dbl | Unix timestamp (milliseconds) of session end |

Visible Issues: The categories of experience is undefined categorical scale. The data may contain self-reported bias.

Potential Issues: The categories of played_hours or age may contain missing value(NA) or extreme value. 

In [20]:
#(2) Questions

broad question: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

specific question: Can we predict whether a player subscribes to the newsletter (subscribe) based on their age, and total hours played in the players.csv dataset?

In my question, the target(Response variable) is subscribe, and the predictor are age and played_hours. The players.csv dataset contains the response variable and the chosen predictors (age and played_hours). First we need to split the data to training dataset and testing dataset. Second we use step_scale() and step_center() to standardize numerical variables(age, played_hours), so that they are on the same scale. Finally, we build a classification model to predict the target variable subscribe based on two predictors.

In [22]:
#(3) Exploratory Data Analysis and Visualization

In [6]:
library(tidyverse)
library(tidymodel)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org



In [7]:
players_url <- "https://raw.githubusercontent.com/Jmmchai/individual-planning-stage/refs/heads/main/players.csv"
sessions_url <- "https://raw.githubusercontent.com/Jmmchai/individual-planning-stage/refs/heads/main/sessions.csv"

players <- read_csv(players_url)
sessions <- read_csv(sessions_url)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [8]:
glimpse(players)
glimpse(sessions)

Rows: 196
Columns: 7
$ experience   <chr> "Pro", "Veteran", "Veteran", "Amateur", "Regular", "Amate…
$ subscribe    <lgl> TRUE, TRUE, FALSE, TRUE, TRUE, TRUE, TRUE, FALSE, TRUE, T…
$ hashedEmail  <chr> "f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8…
$ played_hours <dbl> 30.3, 3.8, 0.0, 0.7, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 1.6, 0…
$ name         <chr> "Morgan", "Christian", "Blake", "Flora", "Kylie", "Adrian…
$ gender       <chr> "Male", "Male", "Male", "Female", "Male", "Female", "Fema…
$ Age          <dbl> 9, 17, 17, 21, 21, 17, 19, 21, 47, 22, 23, 17, 25, 22, 17…
Rows: 1,535
Columns: 5
$ hashedEmail         <chr> "bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8a…
$ start_time          <chr> "30/06/2024 18:12", "17/06/2024 23:33", "25/07/202…
$ end_time            <chr> "30/06/2024 18:24", "17/06/2024 23:46", "25/07/202…
$ original_start_time <dbl> 1.71977e+12, 1.71867e+12, 1.72193e+12, 1.72188e+12…
$ original_end_time   <dbl> 1.71977e+12, 1.71867e+12, 1.72193e+12, 1.72188e+

In [10]:
summary(players)

  experience        subscribe       hashedEmail         played_hours    
 Length:196         Mode :logical   Length:196         Min.   :  0.000  
 Class :character   FALSE:52        Class :character   1st Qu.:  0.000  
 Mode  :character   TRUE :144       Mode  :character   Median :  0.100  
                                                       Mean   :  5.846  
                                                       3rd Qu.:  0.600  
                                                       Max.   :223.100  
                                                                        
     name              gender               Age       
 Length:196         Length:196         Min.   : 9.00  
 Class :character   Class :character   1st Qu.:17.00  
 Mode  :character   Mode  :character   Median :19.00  
                                       Mean   :21.14  
                                       3rd Qu.:22.75  
                                       Max.   :58.00  
                               

In [11]:
summary(sessions)

 hashedEmail         start_time          end_time         original_start_time
 Length:1535        Length:1535        Length:1535        Min.   :1.712e+12  
 Class :character   Class :character   Class :character   1st Qu.:1.716e+12  
 Mode  :character   Mode  :character   Mode  :character   Median :1.719e+12  
                                                          Mean   :1.719e+12  
                                                          3rd Qu.:1.722e+12  
                                                          Max.   :1.727e+12  
                                                                             
 original_end_time  
 Min.   :1.712e+12  
 1st Qu.:1.716e+12  
 Median :1.719e+12  
 Mean   :1.719e+12  
 3rd Qu.:1.722e+12  
 Max.   :1.727e+12  
 NA's   :2          